## **Limpieza del CSV de "Humedad del aire a 2 metros"**
----

#### ***Importación de paquetes***

In [34]:
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

#### ***Cargar el CSV mediante Pandas***

In [35]:
df = pd.read_csv("../CSV/Humedad_del_Aire_2_metros_20240613.csv")
df

C:\Users\marti\AppData\Local\Temp\ipykernel_4912\754846391.py:1: DtypeWarning: Columns (0,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../CSV/Humedad_del_Aire_2_metros_20240613.csv")


,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,57025020,27.0,06/28/2008 04:00:00 PM,88.00000,GORGONA GUAPI,CAUCA,GUAPI,GUAVIARE,2.963000,-78.174,Humedad del aire 2 mt,%
1,53045040,27.0,12/03/2012 01:00:00 AM,96.00000,AEROPUERTO GUAPI - AUT,CAUCA,GUAPI,TAPAJE - DAGUA - DIRECTOS,2.574417,-77.89475,Humedad del aire 2 mt,%
2,21055070,27.0,04/16/2012 08:00:00 PM,85.00000,INZA - AUT,CAUCA,INZÁ,ALTO MAGDALENA,2.548194,-76.063944,Humedad del aire 2 mt,%
3,52025080,27.0,07/15/2014 03:00:00 AM,79.00000,ESTRECHO PATIA - AUT,CAUCA,PATÍA (EL BORDO),PATÍA,1.962778,-77.120278,Humedad del aire 2 mt,%
4,52025501,27.0,05/22/2018 07:55:00 PM,87.13305,LA VEGA - AUT,CAUCA,LA VEGA,PATÍA,2.092942,-76.812028,Humedad del aire 2 mt,%
...,...,...,...,...,...,...,...,...,...,...,...,...
8936971,0053115502,27.0,06/01/2020 12:20:00 PM,89.37495,JULIO FERNANDEZ - AUT,VALLE DEL CAUCA,RESTREPO,TAPAJE - DAGUA - DIRECTOS,3.809164,-76.5330{,NaN,NaN
8936972,"""error"" : true",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8936973,"""message"" : ""Internal error""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8936974,"""status"" : 500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### ***Liberarnos de ciertas columnas***

In [36]:
df = df.drop(["CodigoSensor", "CodigoEstacion", "NombreEstacion", "DescripcionSensor", "UnidadMedida"], axis=1)
df.head()

,FechaObservacion,ValorObservado,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud
0,06/28/2008 04:00:00 PM,88.00000,CAUCA,GUAPI,GUAVIARE,2.963000,-78.174
1,12/03/2012 01:00:00 AM,96.00000,CAUCA,GUAPI,TAPAJE - DAGUA - DIRECTOS,2.574417,-77.89475
2,04/16/2012 08:00:00 PM,85.00000,CAUCA,INZÁ,ALTO MAGDALENA,2.548194,-76.063944
3,07/15/2014 03:00:00 AM,79.00000,CAUCA,PATÍA (EL BORDO),PATÍA,1.962778,-77.120278
4,05/22/2018 07:55:00 PM,87.13305,CAUCA,LA VEGA,PATÍA,2.092942,-76.812028


#### ***Enfocarnos en los departamentos de la región Pacífica***

In [37]:
#df.head()
df["Departamento"].value_counts()

Departamento
VALLE DEL CAUCA    3502675
NARIÑO             2178996
CAUCA              2133539
CHOCO               648614
CHOCÓ               445428
NARINO               27720
Name: count, dtype: int64

#### ***Limpieza en los departamentos***

In [38]:
diccionario_reemplazo = {
    "CHOCO": "CHOCÓ",
    "NARINO": "NARIÑO"
}

df["Departamento"] = df["Departamento"].replace(diccionario_reemplazo)
df["Departamento"].value_counts()

Departamento
VALLE DEL CAUCA    3502675
NARIÑO             2206716
CAUCA              2133539
CHOCÓ              1094042
Name: count, dtype: int64

#### ***Guardar la nueva versión del dataset***

In [39]:
df.to_csv("../HumedadAire_RegionPacifico.csv", index=False)